In [1]:
import numpy as np
import tensorflow as tf
import time
import roslib.packages as rp
pkg_path = rp.get_pkg_dir('prob_grasp_planner')
import sys
sys.path.append(pkg_path + '/src/grasp_active_learning')
from grasp_success_network import GraspSuccessNetwork

In [2]:
update_voxel_enc = False
grasp_net = GraspSuccessNetwork(update_voxel_enc)
is_train = False
grasp_net.grasp_net_train_test(train_mode=is_train)
config_grad = tf.gradients(grasp_net.grasp_net_res['suc_prob'], grasp_net.holder_config)

init = tf.global_variables_initializer()
saver = tf.train.Saver()
tf.get_default_graph().finalize()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use tf.cast instead.


In [6]:
grasp_model_path = pkg_path + '/models/grasp_al_net/grasp_net_freeze_enc.ckpt'
sess = tf.Session()
saver.restore(sess, grasp_model_path)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /home/qingkai/catkin_ws/src/prob_grasp_planner/models/grasp_al_net/grasp_net_freeze_enc.ckpt


In [23]:
def pred_suc(sess, grasp_voxel_grid, grasp_config, grasp_obj_size):
    feed_dict = {
        grasp_net.voxel_ae.is_train: False,
        grasp_net.voxel_ae.partial_voxel_grids: [grasp_voxel_grid],
        grasp_net.is_train: False,
        grasp_net.holder_config: [grasp_config],
        grasp_net.holder_obj_size: [grasp_obj_size],
 }
    [suc_prob] = sess.run([grasp_net.grasp_net_res['suc_prob']], feed_dict=feed_dict)
    return suc_prob[0][0]


In [37]:
def compute_config_grad(sess, grasp_voxel_grid, grasp_config, grasp_obj_size):
    feed_dict = {
        grasp_net.voxel_ae.is_train: False,
        grasp_net.voxel_ae.partial_voxel_grids: [grasp_voxel_grid],
        grasp_net.is_train: False,
        grasp_net.holder_config: [grasp_config],
        grasp_net.holder_obj_size: [grasp_obj_size],
 }
    [suc_prob, config_gradient] = sess.run([grasp_net.grasp_net_res['suc_prob'], config_grad], feed_dict=feed_dict)
    return config_gradient[0][0], suc_prob[0][0]

In [39]:
    def compute_num_grad(func, sess, grasp_voxel_grid, grasp_config, grasp_obj_size):
        eps = 10**-4
        grad = np.zeros(len(grasp_config))
        for i in xrange(len(grasp_config)):
            grasp_config_plus = np.copy(grasp_config)
            grasp_config_plus[i] += eps
            obj_prob_plus = func(sess, grasp_voxel_grid, grasp_config_plus, grasp_obj_size)
            grasp_config_minus = np.copy(grasp_config)
            grasp_config_minus[i] -= eps
            obj_prob_minus = func(sess, grasp_voxel_grid, grasp_config_minus, grasp_obj_size)
            #print 'grasp_config_plus:', grasp_config_plus
            #print 'grasp_config_minus:', grasp_config_minus
            #print 'obj_prob_plus:', obj_prob_plus
            #print 'obj_prob_minus:', obj_prob_minus
            ith_grad = (obj_prob_plus - obj_prob_minus) / (2. * eps)
            grad[i] = ith_grad
        return grad


In [41]:


grasp_voxel_grid = 0.5 * np.ones((32, 32, 32, 1))
grasp_config = 0.5 * np.ones(14)
grasp_obj_size = [0.1, 0.1, 0.1]
suc_prob = pred_suc(sess, grasp_voxel_grid, grasp_config, grasp_obj_size)
print suc_prob
config_gradient, suc_prob = compute_config_grad(sess, grasp_voxel_grid, grasp_config, grasp_obj_size)
print config_gradient, suc_prob
config_num_grad = compute_num_grad(pred_suc, sess, grasp_voxel_grid, grasp_config, grasp_obj_size)
print config_num_grad
grad_diff = config_gradient - config_num_grad
print 'config_gradient:', config_gradient
print 'config_num_grad:', config_num_grad
print 'Gradient difference:', grad_diff
print 'Gradient difference abs mean:', np.mean(abs(grad_diff))
print 'Gradient difference abs mean percentage:',  np.mean(abs(grad_diff)) / np.mean(abs(config_gradient))

# grasp_net_test_freeze_path = '/home/qingkai/test_grasp_net/grasp_net_test_freeze.ckpt'
# full_voxel = np.ones((1, 32, 32, 32, 1))
# with tf.Session() as sess:
#     sess.run(init)
#     saver.restore(sess, grasp_net_test_freeze_path)

1.1754088e-10
[ 2.2110175e-10 -3.0834807e-10 -1.3220501e-10 -1.2286608e-10
  1.7196557e-10  1.7203154e-10  2.2509415e-10  2.7675262e-10
  8.7007172e-11  4.5518520e-11 -3.3434464e-11 -5.6859809e-11
  8.5369704e-11  3.4791104e-11] 1.1754088e-10
[ 2.20795604e-10 -3.08295056e-10 -1.32255318e-10 -1.23304145e-10
  1.72639680e-10  1.72639680e-10  2.24126273e-10  2.75751644e-10
  8.63198402e-11  4.71150896e-11 -3.24740235e-11 -5.82867088e-11
  8.40993941e-11  3.46944695e-11]
config_gradient: [ 2.2110175e-10 -3.0834807e-10 -1.3220501e-10 -1.2286608e-10
  1.7196557e-10  1.7203154e-10  2.2509415e-10  2.7675262e-10
  8.7007172e-11  4.5518520e-11 -3.3434464e-11 -5.6859809e-11
  8.5369704e-11  3.4791104e-11]
config_num_grad: [ 2.20795604e-10 -3.08295056e-10 -1.32255318e-10 -1.23304145e-10
  1.72639680e-10  1.72639680e-10  2.24126273e-10  2.75751644e-10
  8.63198402e-11  4.71150896e-11 -3.24740235e-11 -5.82867088e-11
  8.40993941e-11  3.46944695e-11]
Gradient difference: [ 3.06143999e-13 -5.30131494e